In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as f
from pyspark.sql import types as t
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder.appName('lol').getOrCreate()

In [3]:
pdf = pd.read_csv('./train.csv')

In [4]:
target = 'Survived'

In [5]:
converter = {'int64':t.IntegerType(),'float64':t.FloatType(),'object':t.StringType()}

In [6]:
schema = t.StructType([t.StructField(c,converter[str(i[0])],False) for c, i in pdf.dtypes.to_frame().iterrows()])

In [7]:
df = spark.createDataFrame(pdf, schema=schema)

In [15]:
df.fillna(0).registerTempTable('titanic')

In [18]:
df_new = df.select(['age', 'parch', 'pclass'])

In [55]:
m = np.array([[1, 0],
          [0, 1],
          [5, 2]])

In [24]:
isinstance(df, DataFrame)

True

In [25]:
m[1]

(3, 2)

In [28]:
D, K = m.shape
D_df = len(df_new.columns)
assert(D==D_df) 

In [50]:
def matrix_multiply(df, m, columns=None):
    
    D, K = m.shape
    D_df = len(df.columns)
    
    if D != D_df:
        raise RuntimeError('Inner dimensions do not match (spark_df = {}, np_matrix = {})'.format(D_df,D))
    
    new_cols = []
    for i in range(K):
        new_col = []
        for j in range(D):
            new_col.append('{}*{}'.format(m[j,i],df.columns[j]))
        new_cols.append(' + '.join(new_col) + ' as col_%s' %i)
    columns = ',\n\t'.join(new_cols)
    df.registerTempTable('table')
    
    query = 'select\n\t{}\nfrom\n\t{}'.format(columns, 'table')

    return query

In [51]:
df_new.show()

+----+-----+------+
| age|parch|pclass|
+----+-----+------+
|22.0|    0|     3|
|38.0|    0|     1|
|26.0|    0|     3|
|35.0|    0|     1|
|35.0|    0|     3|
| NaN|    0|     3|
|54.0|    0|     1|
| 2.0|    1|     3|
|27.0|    2|     3|
|14.0|    0|     2|
| 4.0|    1|     3|
|58.0|    0|     1|
|20.0|    0|     3|
|39.0|    5|     3|
|14.0|    0|     3|
|55.0|    0|     2|
| 2.0|    1|     3|
| NaN|    0|     2|
|31.0|    0|     3|
| NaN|    0|     3|
+----+-----+------+
only showing top 20 rows



In [56]:
spark.sql(matrix_multiply(df_new.fillna(0), m)).show()

+-----+-----+
|col_0|col_1|
+-----+-----+
| 37.0|  6.0|
| 43.0|  2.0|
| 41.0|  6.0|
| 40.0|  2.0|
| 50.0|  6.0|
| 15.0|  6.0|
| 59.0|  2.0|
| 17.0|  7.0|
| 42.0|  8.0|
| 24.0|  4.0|
| 19.0|  7.0|
| 63.0|  2.0|
| 35.0|  6.0|
| 54.0| 11.0|
| 29.0|  6.0|
| 65.0|  4.0|
| 17.0|  7.0|
| 10.0|  4.0|
| 46.0|  6.0|
| 15.0|  6.0|
+-----+-----+
only showing top 20 rows



In [57]:
spark.sql('select sigmoid(age), parch, pclass from titanic limit 10').show()

AnalysisException: "Undefined function: 'sigmoid'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 7"

In [8]:
df

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: float, SibSp: int, Parch: int, Ticket: string, Fare: float, Cabin: string, Embarked: string]